In [177]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time
import os
import urllib
import re
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [47]:
# Automating the browser

driver = webdriver.Chrome("chromedriver.exe")

url = "https://www.amazon.in/"

# opending brower

driver.get(url) # we are opening the url which is saved in the url variable 
driver.maximize_window() # it will maximize the website page 

# user will provide input 
user_input = input("Enter the product name :- ") # by using this code, program will ask user to enter the product name

# automate the search bar
search_bar= driver.find_element_by_id("twotabsearchtextbox") # it the search bar ID,which will help to automate the search bar
search_bar.send_keys(user_input) # send_keys is the functon which take user_input to the website search bar
time.sleep(2) # we want our program to be stopped for 2 sec

# click on search

driver.find_element_by_id("nav-search-submit-button").click() # automating the the submit button, once user input goes to search bar, it will wait for 2 sec and then click on submit button  
time.sleep(2) # after clicking the submit button program to be stopped for 2 sec


# continue second question



Enter the product name :- guitars


In [48]:
# this question will follow the second question in the same sequence, we will extract the url of a product and from the product will extract required information and details.

product_url = [] # creating empty list and will save all the url in this list
for i in range(3): # we are asked to scrap first 3 pages thus loop is run till 3 time
    urls = driver.find_elements_by_xpath("//h2['class=a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a") # xpath for urls
    time.sleep(2) # program will stop for 2 sec
    for j in urls: # this loop will itterate over urls and extract url and append in product url list and program will rest for 2 sec
        product_url.append(j.get_attribute("href"))
        
    time.sleep(2)
    # written next button code to jump to next page 
    driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']").click()
    time.sleep(2)
        
        


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [89]:
# in this question we will extract the information from the urls which are extracted in question one
# created number of emtpy list for saving the scrap information 
Brand_Name =[]
Name_of_the_Product =[] 
Price =[] 
Return_Exchange =[]
Expected_Delivery =[]
Availability =[]

for i in product_url:
    driver.get(i)
    time.sleep(2)
    
#     Brand_name
    try:
        brand_name = driver.find_element_by_xpath("//div[@class='a-expander-content a-expander-section-content a-section-expander-inner']/table/tbody/tr[1]/td")
        Brand_Name.append(brand_name.text)
        time.sleep(3)
    except:
        Brand_Name.append("-")
        
#     Name_of_the_Product

    try:
        product_name = driver.find_element_by_id("productTitle")
        Name_of_the_Product.append(product_name.text)
    except:
        Name_of_the_Product.append("-")
        
          
    #Price 
    
    try:
        price=driver.find_element_by_xpath("//span[@class='a-price a-text-price a-size-base']/span[2]")
        Price.append(price.text)
    except:
        Price.append("-")
    
    # Return_Exchange
    try:
        return_exchange = driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']/span/div[2]/a")
        Return_Exchange.append(return_exchange.text)
        time.sleep(3)
    except:
        Return_Exchange.append("-")
    
   #Expected_Delivery

    try:
        delivery =driver.find_element_by_xpath("//div[@id='mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE']/b")
        Expected_Delivery.append(delivery.text)
    except:
        Expected_Delivery.append("-")
        
    # Availability
    
    try:
        prod_status = driver.find_element_by_xpath("//th[contains(text(),'Date First Available')]/../td")
        Availability.append(prod_status.text.strip())
    except:
        Availability.append("-")
        
        

products = pd.DataFrame({})
products['Brand']=Brand_Name
products['Product_Name']=Name_of_the_Product
products['Price']=Price
products['Return_Exchange']=Return_Exchange
products['Expected_Delivery']=Expected_Delivery
products['Availability']=Availability
products['product_url']=product_url

driver.close() # closing chrome driver

In [91]:
products
#print(len(Brand_Name))
#print(len(Name_of_the_Product))
#print(len(Price))
#print(len(Return_Exchange))
#print(len(Expected_Delivery))
#print(len(Availability))
#print(len(product_url))


,Brand,Product_Name,Price,Return_Exchange,Expected_Delivery,Availability,product_url
0,Kadence,Kadence Frontier guitar with Online Guitar lea...,"₹6,050.00",7 Days Replacement,"Friday, May 20",30 May 2016,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,"₹8,580.00",7 Days Replacement,"Monday, May 23",29 June 2016,https://www.amazon.in/gp/slredirect/picassoRed...
2,VAULT,Vault Traveller 34 Inch Acoustic Guitar With B...,-,7 Days Replacement,"Tuesday, May 24",17 September 2019,https://www.amazon.in/gp/slredirect/picassoRed...
3,73 x 25 x 15 cm; 1.8 Kilograms,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",-,7 Days Replacement,"Saturday, May 21",6 March 2019,https://www.amazon.in/gp/slredirect/picassoRed...
4,Intern,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",-,7 Days Replacement,"Saturday, May 21",30 June 2017,https://www.amazon.in/Intern-INT-38C-Acoustic-...
...,...,...,...,...,...,...,...
169,108 x 45.01 x 10.01 cm; 3.4 Kilograms,"Yamaha PACIFICA012 Electric Guitar, Red Metallic",-,7 Days Replacement,"Monday, May 23",20 April 2015,https://www.amazon.in/Yamaha-PAC012-Electric-G...
170,Pyle,"Beginner 30"" Classical Acoustic Guitar - 6 Str...",-,7 Days Replacement,-,11 January 2020,https://www.amazon.in/Classical-Traditional-Fr...
171,PERLMAN,"Perlman Guitar Fretboard, Fingerboard Stickers...",-,7 Days Replacement,"Saturday, May 21",10 June 2021,https://www.amazon.in/Perlman-Fretboard-Finger...
172,GIVSON,GIVSON Ketostics Venus Special Guitar (black) ...,-,7 Days Replacement,"Sunday, May 22",12 June 2019,https://www.amazon.in/Givson-Special-Guitar-VS...


# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [98]:
driver = webdriver.Chrome("chromedriver.exe") # chrome driver is required to automate the chrome browser
url = "https://images.google.com/" 

# opending brower
driver.get(url) 

# folder in which photos will be saved

save_folder = "Fruit image" # folder name
os.makedirs(save_folder)

search_box = driver.find_element_by_xpath("//input[@class ='gLFyf gsfi']") # search box xpath for accessing the search bar
search_box.send_keys('fruits') # send_keys will take user input to search bar
time.sleep(2) # program will stop for 1 sec

driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").click() # automating the submit button, once input will be given to search bar, program will stop for 1 sec and then click 
time.sleep(1)

# this below mentioned code will scroll the the page down  
value = 0
for i in range(2):
    driver.execute_script("scrollBy("+str(value)+",+1000);")
    value +=1000
    time.sleep(1)
    
image_url = driver.find_elements_by_xpath("//div[@class ='bRMDJf islir']/img") # saving images url in the image_url variable
"""Running for loop for extracting the images"""
url = []
for i in range(0,10):
    url =(image_url[i].get_attribute("src"))
    try:
        urllib.request.urlretrieve(url,os.path.join(save_folder,'fruits'+str(i)+'.jpg'))
    except Exception as e:
        print(e)    
        
driver.close() # closing chrome driver

Scrapping for Cars

In [114]:
driver = webdriver.Chrome("chromedriver.exe") # chrome driver is required to automate the chrome browser
url = "https://images.google.com/" 

# opending brower
driver.get(url) 

# folder in which photos will be saved

save_folder = "cars" # folder name
os.makedirs(save_folder)

search_box = driver.find_element_by_xpath("//input[@class ='gLFyf gsfi']") # search box xpath for accessing the search bar
search_box.send_keys('cars') # send_keys will take user input to search bar
time.sleep(5) # program will stop for 1 sec

driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").submit() # automating the submit button, once input will be given to search bar, program will stop for 1 sec and then click 
time.sleep(1)

# this below mentioned code will scroll the the page down  
value = 0
for i in range(2):
    driver.execute_script("scrollBy("+str(value)+",+1000);")
    value +=1000
    time.sleep(1)
    
image_url = driver.find_elements_by_xpath("//div[@class ='bRMDJf islir']/img") # saving images url in the image_url variable
"""Running for loop for extracting the images"""
url = []
for i in range(0,10):
    url =(image_url[i].get_attribute("src"))
    try:
        urllib.request.urlretrieve(url,os.path.join(save_folder,'cars'+str(i)+'.jpg'))
    except Exception as e:
        print(e)   
        
driver.close() # closing chrome driver

Scrapping for Machine Learning

In [115]:
driver = webdriver.Chrome("chromedriver.exe") # chrome driver is required to automate the chrome browser
url = "https://images.google.com/" 

# opending brower
driver.get(url) 

# folder in which photos will be saved

save_folder = "Machine Learning" # folder name
os.makedirs(save_folder)

search_box = driver.find_element_by_xpath("//input[@class ='gLFyf gsfi']") # search box xpath for accessing the search bar
search_box.send_keys('Machine Learning') # send_keys will take user input to search bar
time.sleep(5) # program will stop for 1 sec

driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").submit() # automating the submit button, once input will be given to search bar, program will stop for 1 sec and then click 
time.sleep(1)

# this below mentioned code will scroll the the page down  
value = 0
for i in range(2):
    driver.execute_script("scrollBy("+str(value)+",+1000);")
    value +=1000
    time.sleep(1)
    
image_url = driver.find_elements_by_xpath("//div[@class ='bRMDJf islir']/img") # saving images url in the image_url variable
"""Running for loop for extracting the images"""
url = []
for i in range(0,10):
    url =(image_url[i].get_attribute("src"))
    try:
        urllib.request.urlretrieve(url,os.path.join(save_folder,'ML'+str(i)+'.jpg'))
    except Exception as e:
        print(e)   
        
driver.close() # closing chrome driver

Scrapping for Guitars

In [116]:
driver = webdriver.Chrome("chromedriver.exe") # chrome driver is required to automate the chrome browser
url = "https://images.google.com/" 

# opending brower
driver.get(url) 

# folder in which photos will be saved

save_folder = "Guitars" # folder name
os.makedirs(save_folder)

search_box = driver.find_element_by_xpath("//input[@class ='gLFyf gsfi']") # search box xpath for accessing the search bar
search_box.send_keys('Guitars') # send_keys will take user input to search bar
time.sleep(5) # program will stop for 1 sec

driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").submit() # automating the submit button, once input will be given to search bar, program will stop for 1 sec and then click 
time.sleep(1)

# this below mentioned code will scroll the the page down  
value = 0
for i in range(2):
    driver.execute_script("scrollBy("+str(value)+",+1000);")
    value +=1000
    time.sleep(1)
    
image_url = driver.find_elements_by_xpath("//div[@class ='bRMDJf islir']/img") # saving images url in the image_url variable
"""Running for loop for extracting the images"""
url = []
for i in range(0,10):
    url =(image_url[i].get_attribute("src"))
    try:
        urllib.request.urlretrieve(url,os.path.join(save_folder,'Guitars'+str(i)+'.jpg'))
    except Exception as e:
        print(e)   
        
driver.close() # closing chrome driver

Scrapping for Cakes

In [117]:
driver = webdriver.Chrome("chromedriver.exe") # chrome driver is required to automate the chrome browser
url = "https://images.google.com/" 

# opending brower
driver.get(url) 

# folder in which photos will be saved

save_folder = "Cakes" # folder name
os.makedirs(save_folder)

search_box = driver.find_element_by_xpath("//input[@class ='gLFyf gsfi']") # search box xpath for accessing the search bar
search_box.send_keys('Cakes') # send_keys will take user input to search bar
time.sleep(5) # program will stop for 1 sec

driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']").submit() # automating the submit button, once input will be given to search bar, program will stop for 1 sec and then click 
time.sleep(1)

# this below mentioned code will scroll the the page down  
value = 0
for i in range(2):
    driver.execute_script("scrollBy("+str(value)+",+1000);")
    value +=1000
    time.sleep(1)
    
image_url = driver.find_elements_by_xpath("//div[@class ='bRMDJf islir']/img") # saving images url in the image_url variable
"""Running for loop for extracting the images"""
url = []
for i in range(0,10):
    url =(image_url[i].get_attribute("src"))
    try:
        urllib.request.urlretrieve(url,os.path.join(save_folder,'Cakes'+str(i)+'.jpg'))
    except Exception as e:
        print(e)   
        
driver.close() # closing chrome driver

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [147]:
# automating browser and opening website

driver = webdriver.Chrome("chromedriver.exe")

driver.get("https://www.flipkart.com/")
time.sleep(3)

# when we open flipkart, it asks for login details and we have to skip that. so below xpath will cancel it
try:
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click() 
except:
    driver.find_element_by_xpath("//div[@class='_2QfC02']/button[@class='_2KpZ6l _2doB4z']").click()
    

# automating search bar, user will provide input, send_keys will take user input to the search bar and search it.
user_search = input("Enter the product name:- ")
search_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
search_bar.send_keys(user_search)
time.sleep(2)

# Submit button , will search the item
driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click() #clicking on submit button 
time.sleep(2)

# extracting the urls of the product and with these urls we will get the required information
url = driver.find_elements_by_xpath("//a[@class='_1fQZEK']") # extracting URls
Product_url = [i.get_attribute("href") for i in url]


# creating empty lists for saving the details

Brand_name =[]
Smartphone_name =[]
color =[]
RAM = []
ROM =  []
Primary_camera = []
Secondary_cam = []
Display_size = []
Battery_capa = []
Price = []

""" Running a loop for extracting all the information"""

for i in Product_url:
    driver.get(i)
    time.sleep(2)
    
    # brand name
    try:
        brand = driver.find_element_by_xpath("//h1[@class ='yhB1nd']/span")
        Brand_name.append(brand.text.split()[0])
    except:
        Brand_name.append("NA")
    
    # Price
    try:
        price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        Price.append(price.text)
        time.sleep(2)
    except:
        Price.append("NA")
        
#     we have a button names "readfull", while opening it specification will expand and we can see all details.
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    time.sleep(2)
    
    # Smartphone_name
    try:
        phone_name = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[3]/td[2]/ul/li")
        Smartphone_name.append(phone_name.text)
    except:
        Smartphone_name.append("NA")
    
     # phone color   
    try:
        Color = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[1]/table/tbody/tr[4]/td[2]/ul/li")
        color.append(Color.text)
    except:
        color.append("NA")
    
    # Ram status
    try:
        ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[2]/td[2]/ul/li")
        RAM.append(ram.text)
    except:
        RAM.append("NA")
    
    #ROM Status
    try:
        rom = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[1]/td[2]/ul/li")
        ROM.append(ram.text)
    except:
        ROM.append("NA")
        
    # Primany Camera
    try:
        p_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[2]/td[2]/ul/li")
        Primary_camera.append(p_cam.text)
    except:
        Primary_camera.append("NA")
        
    # secondary cam
    try:
        s_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[5]/td[2]/ul/li")
        Secondary_cam.append(s_cam.text)
    except:
        Secondary_cam.append("NA")

    #  Display_size  
    try:
        display_size = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[1]/td[2]/ul/li")
        Display_size.append(display_size.text)
    except:
        Display_size.append("NA")
        
           
    # Battery_capa
    try:
        battery = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/table/tbody/tr[1]/td[2]/ul/li")
        Battery_capa.append(battery.text)
    except:
        Battery_capa.append("NA")
        
driver.close() # closing chrome driver

Enter the product name:- pixel 4a


In [153]:
df = pd.DataFrame({"Brand_name":Brand_name,"Smartphone_name":Smartphone_name,"color":color,"RAM":RAM,"ROM":ROM,
                               "Primary_camera":Primary_camera,"Secondary_cam":Secondary_cam,"Display_size":Display_size,"Battery_capa":Battery_capa,"Price":Price,"Product_url":Product_url})
df

,Brand_name,Smartphone_name,color,RAM,ROM,Primary_camera,Secondary_cam,Display_size,Battery_capa,Price,Product_url
0,Google,Pixel 4a,Just Black,6 GB,6 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),69.4 mm,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,vivo,T1 44W,Ice Dawn,8 GB,8 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹17,999",https://www.flipkart.com/vivo-t1-44w-ice-dawn-...
2,vivo,T1 44W,Midnight Galaxy,6 GB,6 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹15,999",https://www.flipkart.com/vivo-t1-44w-midnight-...
3,vivo,T1 44W,Ice Dawn,6 GB,6 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹15,999",https://www.flipkart.com/vivo-t1-44w-ice-dawn-...
4,vivo,T1 44W,Midnight Galaxy,4 GB,4 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-midnight-...
5,vivo,T1 44W,Starry Sky,4 GB,4 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-starry-sk...
6,vivo,T1 44W,Starry Sky,8 GB,8 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹17,999",https://www.flipkart.com/vivo-t1-44w-starry-sk...
7,vivo,T1 44W,Starry Sky,6 GB,6 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹15,999",https://www.flipkart.com/vivo-t1-44w-starry-sk...
8,vivo,T1 44W,Midnight Galaxy,8 GB,8 GB,50MP + 2MP + 2MP,16MP Front Camera,16.36 cm (6.44 inch),5000 mAh,"₹17,999",https://www.flipkart.com/vivo-t1-44w-midnight-...
9,Redmi,Redmi 9A Sport,Coral Green,2 GB,2 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹7,067",https://www.flipkart.com/redmi-9a-sport-coral-...


In [141]:
df.to_csv('mobile_info.csv') # saving in csv file

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [128]:
# automate the browser and opening site 

driver = webdriver.Chrome("chromedriver.exe")
url = "https://www.google.com/maps"
driver.get(url)
time.sleep(2)
# write in search box

user_input = input("Enter the city name:- ") # user input, code will ask for input from user
search_bar = driver.find_element_by_id("searchboxinput") # xpath for search box
search_bar.send_keys(user_input) # sending user input to searchbar 
time.sleep(2)

# submit_button

driver.find_element_by_id("searchbox-searchbutton").click() # click on search button to search item
time.sleep(2)

# fecting current URL from the webpage to extract the desire output  
current_page_url = driver.current_url # 
print(current_page_url) # printing the url

# extracting the lat and log from the url by using the reges, \d+.\d+ it will match all the number and extract it
location_coordinates = re.findall('(\d+.\d+)',current_page_url) # it will result a comma seprated list and by sliceing we can extract the required information  

# printing the lat and log 
print("*"*70) # printing star just to seprate url and end result
print("Location coordinates")
print(user_input,"Latitude",location_coordinates[0]) # at 0th index , we have attitude and at 1st index we have longitude
print(user_input,"Longitude",location_coordinates[1])

driver.close() # closing chrome driver

Enter the city name:- Delhi
https://www.google.com/maps/place/Delhi/@28.6992141,77.4128081,15z/data=!4m5!3m4!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902
**********************************************************************
Location coordinates
Delhi Latitude 28.6992141
Delhi Longitude 77.4128081


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [138]:
# automate browser
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://trak.in/")
time.sleep(2)

# opening funding details tab

funding_details = driver.find_element_by_xpath("//li[@id='menu-item-51510']/a").get_attribute("href")
driver.get(funding_details)
time.sleep(2)

# selecting different number of rows 
drop_down = Select(driver.find_element_by_xpath("//div[@id='tablepress-56_length']/label/select"))
time.sleep(2)
drop_down.select_by_value("100") # i am selecting the drop down from value, we can select it from index or visible text options also

# it will be used for printing the header 
cols = driver.find_elements_by_xpath("//*[@id='tablepress-56_wrapper']/div[3]/div[1]/div/table/thead/tr/th") 

# we are taking the lenght of row and col, so that we can run loop until rows and cols and we can fetch the data accordingly 
rows = len(driver.find_elements_by_xpath("//*[@id='tablepress-56']/tbody/tr"))
col = len(driver.find_elements_by_xpath("//*[@id='tablepress-56_wrapper']/div[3]/div[1]/div/table/thead/tr/th"))

print("**************************** March 2021****************************** \n")

for i in cols:
    print(i.text,end =" ")
    
print("\n") # giving a line 
"""i have created two loops one for rows and second for columns,it will run untill last row. have defined xpath in the xpath
i have passed row and cols as paraeter and extracting the information one by one"""

for r in range(1,rows+1):
    for c in range(1,col+1):
        # as menioned passing rows and col in the xpath converting number into string
        value = driver.find_element_by_xpath("//*[@id='tablepress-56']/tbody/tr["+str(r)+"]/td["+str(c)+"]").text
        print(value, end = " ") # printing all the value line by line
    print() # this prinint is helping to print the next line from
    
    

#Extracting data for Feb 2021
    
# selecting different number of rows 
drop_down = Select(driver.find_element_by_xpath("//div[@id='tablepress-55_length']/label/select"))
time.sleep(2)
drop_down.select_by_value("100") # i am selecting the drop down from value, we can select it from index or visible text options also

# it will be used for printing the header 
cols = driver.find_elements_by_xpath("//*[@id='tablepress-55_wrapper']/div[3]/div[1]/div/table/thead/tr/th") 

# we are taking the lenght of row and col, so that we can run loop until rows and cols and we can fetch the data accordingly 
rows = len(driver.find_elements_by_xpath("//*[@id='tablepress-55']/tbody/tr"))
col = len(driver.find_elements_by_xpath("//*[@id='tablepress-55_wrapper']/div[3]/div[1]/div/table/thead/tr/th"))

print("\n\n**************************** Feb 2021****************************** \n")

for i in cols:
    print(i.text,end =" ")
    
print("\n") # giving a line 
"""i have created two loops one for rows and second for columns,it will run untill last row. have defined xpath in the xpath
i have passed row and cols as paraeter and extracting the information one by one"""

for r in range(1,rows+1):
    for c in range(1,col+1):
        # as menioned passing rows and col in the xpath converting number into string
        value = driver.find_element_by_xpath("//*[@id='tablepress-55']/tbody/tr["+str(r)+"]/td["+str(c)+"]").text
        print(value, end = " ") # printing all the value line by line
    print() # this prinint is helping to print the next line from
    
    
    
#Extracting data for Jan 2021
    
# selecting different number of rows 
drop_down = Select(driver.find_element_by_xpath("//div[@id='tablepress-54_length']/label/select"))
time.sleep(2)
drop_down.select_by_value("100") # i am selecting the drop down from value, we can select it from index or visible text options also

# it will be used for printing the header 
cols = driver.find_elements_by_xpath("//*[@id='tablepress-54_wrapper']/div[3]/div[1]/div/table/thead/tr/th") 

# we are taking the lenght of row and col, so that we can run loop until rows and cols and we can fetch the data accordingly 
rows = len(driver.find_elements_by_xpath("//*[@id='tablepress-54']/tbody/tr"))
col = len(driver.find_elements_by_xpath("//*[@id='tablepress-54_wrapper']/div[3]/div[1]/div/table/thead/tr/th"))

print("\n\n**************************** Jan 2021****************************** \n")

for i in cols:
    print(i.text,end =" ")
    
print("\n") # giving a line 
"""i have created two loops one for rows and second for columns,it will run untill last row. have defined xpath in the xpath
i have passed row and cols as paraeter and extracting the information one by one"""

for r in range(1,rows+1):
    for c in range(1,col+1):
        # as menioned passing rows and col in the xpath converting number into string
        value = driver.find_element_by_xpath("//*[@id='tablepress-54']/tbody/tr["+str(r)+"]/td["+str(c)+"]").text
        print(value, end = " ") # printing all the value line by line
    print() # this prinint is helping to print the next line from
    
    
driver.close() # closing chrome driver

**************************** March 2021****************************** 

Sr. No. Date (dd/mm/yyyy) Startup Name Industry / Vertical Sub-Vertical City / Location Investors' Name Investment Type Amount (In USD) 

1 04/03/2021 DealShare E-commerce Online shopping platform Jaipur, Rajasthan Innoven Capital Debt Financing 250,000,000 
2 31/03/2021 Uniphore Technology Conversational Service Automation (CSA) Palo Alto Sorenson Capital Partners Series D 140,000,000 
3 30/03/2021 Dunzo E-commerce Hyper-local delivery app Bengaluru Krishtal Advisors Pte Ltd Series E 8,000,000 
4 30/03/2021 BYJU’S Edu-tech Online tutoring Bengaluru MC Global Edtech, B Capital, Baron, others Series F 460,000,000 
5 23/03/2021 SkilloVilla Edu-tech Career and job-oriented upskilling. Bengaluru Titan Capital, others Seed 300,000,000 
6 25/03/2021 CityMall E-commerce Social ecommerce and online grocery platform Gurgaon Accel Partners Series A 11,000,000 
7 26/03/2021 DotPe FinTech Commerce and payments platform to offl

# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [197]:
driver = webdriver.Chrome("chromedriver.exe") 
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

all_laptop_title = driver.find_elements_by_xpath('//div[@data-cat="Top_Ten_En_Scroll"]/a')

all_name = []
for one_title in all_laptop_title:
    all_name.append(one_title.text)
    
all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')

all_spec_list = []
for one_spec in all_specs:

    os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
    display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
    processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
    memory = one_spec.find_element_by_xpath('.//td[text()="Memory"]/parent::*/td[3]').text
    weight = one_spec.find_element_by_xpath('.//td[text()="Weight"]/parent::*/td[3]').text
    dimension = one_spec.find_element_by_xpath('.//td[text()="Dimension"]/parent::*/td[3]').text
    graphics_processor = one_spec.find_element_by_xpath('.//td[text()="Graphics Processor"]/parent::*/td[3]').text
    
    all_spec_list.append({
        'os' : os_name,
        'display' : display,
        'processor' : processor,
        'memory' : memory,
        'weight' : weight,
        'dimension' : dimension,
        'graphics_processor' : graphics_processor,
    })
    
df = pd.DataFrame(all_spec_list)

        
driver.close() # closing chrome driver 

In [198]:
df['Names']=all_name
df

,os,display,processor,memory,weight,dimension,graphics_processor,Names
0,Windows 11 Home,"17"" (3840 x 2160)",12th Gen Intel Core i9-12900HK | 5 GHz,2 TB SSD/16 GBGB DDR5,2.9,397 x 284 x 26,NVIDIA GeForce RTX 3080Ti,MSI RAIDER GE76
1,Windows 11 Home,"15.6"" (2560 x 1440)",12th Gen Intel Core i9-12900H | 2.5 GHz,2 TB SSD/32 GBGB DDR5,2.3,259 x 354 x 27,NVIDIA GeForce RTX 3070 Ti,ASUS ROG STRIX SCAR 15
2,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,ACER NITRO 5
3,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,MSI STEALTH 15M
4,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,ASUS ROG STRIX SCAR 15
5,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,ASUS ROG STRIX SCAR 15
6,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,ASUS ZEPHYRUS G14
7,Windows 11 Home,"16.1"" (1920 x 1080)",12th Gen Intel Core i7-12700H | 4.7 GHz,1 TB SSD/16 GBGB DDR4,2.32,36.92 x 24.8 x 2.3,NVIDIA GeForce RTX 3060,HP OMEN 16
8,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,ASUS ROG ZEPHYRUS DUO 15
9,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,ACER ASPIRE 7 GAMING LAPTOP


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [169]:
driver = webdriver.Chrome("chromedriver.exe") 
driver.get('https://www.forbes.com/billionaires/')
time.sleep(7)
rank=[]
name=[]
net_worth=[]
age=[]
citizenship=[]
source=[]
industry=[]
all_boxes = driver.find_elements_by_xpath('//div[contains(@class,"table-row ")]')
for one_box in all_boxes:
    rank.append(one_box.find_element_by_xpath('.//div[@class="rank"]').text)
    name.append(one_box.find_element_by_xpath('.//div[@class="personName"]').text)
    net_worth.append(one_box.find_element_by_xpath('.//div[@class="netWorth"]').text)
    age.append(one_box.find_element_by_xpath('.//div[@class="age"]').text)
    citizenship.append(one_box.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text)
    source.append(one_box.find_element_by_xpath('.//div[@class="source"]').text)
    industry.append(one_box.find_element_by_xpath('.//div[@class="category"]').text)

        
        
df = pd.DataFrame({"rank":rank,"name":name,"net_worth":net_worth,"age":age,"citizenship":citizenship,"source":industry,"industry":industry})
df
driver.close() # closing chrome driver

,rank,name,net_worth,age,citizenship,source,industry
0,1.,Elon Musk,$219 B,50,United States,Automotive,Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Technology,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,Fashion & Retail,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Technology,Technology
4,5.,Warren Buffett,$118 B,91,United States,Finance & Investments,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,Food & Beverage,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,Finance & Investments,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Technology,Technology
198,197.,David Geffen,$10 B,79,United States,Media & Entertainment,Media & Entertainment


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [173]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.youtube.com/watch?v=YrUqw7uspKg')
# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')
    
    time.sleep(1)
    
    if len(all_box) > 500:
        break
        
all_list = []
for one_box in all_box:
    comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
    upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
    time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text
    
    all_list.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
df = pd.DataFrame(all_list)
df        

,comment,upvote,time
0,Who else knew about this song before shershah?...,2.8K,8 months ago (edited)
1,What I have observed in Punjabi Songs is that ...,12,2 days ago
2,Five years but it still feels that it was rele...,365,3 months ago
3,"Here again,, no matter how old is this song......",91,1 month ago (edited)
4,Happy birthday B praak ji \nYour voice is like...,357,3 months ago
...,...,...,...
515,Love from pakistan like ur sound,1,2 weeks ago
516,I'm one of those few people who have been list...,236,6 months ago
517,Heart touching song,1,4 months ago
518,,,3 months ago


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [187]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3')

final_list = []
page_urls = []

all_boxs = driver.find_elements_by_xpath('//div[@class="property-card" and @data-v-38030316=""]')
for one_box in all_boxs:
    hostel_name = one_box.find_element_by_xpath('.//h2/a').text
    distance_from_city = one_box.find_element_by_xpath('.//a/span').text
    try:
        ratings = one_box.find_element_by_xpath('.//div[contains(@class,"score")]').text
    except:
        ratings = 'No rating'

    total_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"reviews")]').text
    overall_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"keyword")]').text
    privates_from_price = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[0].text
    dorms_from_price  = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[1].text
    facilities = one_box.find_element_by_xpath('.//div[contains(@class,"facilities")]').text.split('\n')

    page_url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
    page_urls.append(page_url)

    final_list.append({
        'hostel_name' : hostel_name,
        'distance_from_city' : distance_from_city,
        'ratings' : ratings,
        'total_reviews' : total_reviews,
        'overall_reviews' : overall_reviews,
        'privates_from_price' : privates_from_price,
        'dorms_from_price' : dorms_from_price,
        'facilities' : facilities,

    })
        
            

df = pd.DataFrame(final_list)
df

,hostel_name,distance_from_city,ratings,total_reviews,overall_reviews,privates_from_price,dorms_from_price,facilities
0,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre,9.0,15337 Total Reviews,Superb,No Privates Available,Dorms From\nRs6512,"[Free WiFi, Follows Covid-19 sanitation guidance]"
1,Selina Camden,Hostel - 5.5km from city centre,9.3,23 Total Reviews,Superb,Privates From\nRs34115,No Dorms Available,[Free WiFi]
2,Generator London,Hostel - 3km from city centre,7.7,6927 Total Reviews,Very Good,Privates From\nRs14648,Dorms From\nRs5435,"[Free WiFi, Follows Covid-19 sanitation guidance]"
3,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,7.4,4229 Total Reviews,Very Good,No Privates Available,Dorms From\nRs4944,"[Free WiFi, Follows Covid-19 sanitation guidance]"
4,Wombat's City Hostel London,Hostel - 3.6km from city centre,8.9,13586 Total Reviews,Fabulous,Privates From\nRs22895.12 Rs17171,No Dorms Available,"[Free WiFi, Follows Covid-19 sanitation guidance]"
5,Saint James Backpackers,Hostel - 5.5km from city centre,6.8,1789 Total Reviews,Good,Privates From\nRs25800,Dorms From\nRs6650,"[Free WiFi, Free Breakfast, Follows Covid-19 s..."
6,Britannia Inn Hotel,Hotel - 14.5km from city centre,8.1,19 Total Reviews,Fabulous,Privates From\nRs9637,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g..."
7,London House Hotel,Hotel - 5.3km from city centre,8.6,1381 Total Reviews,Fabulous,Privates From\nRs35907.74 Rs25135,No Dorms Available,[Free WiFi]
8,Cranbrook Hotel,Hotel - 14.8km from city centre,No rating,58 Total Reviews,No Rating,Privates From\nRs11468,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g..."
9,Belgrave Hotel London,Hotel - 2.2km from city centre,4.6,6 Total Reviews,Rating,Privates From\nRs14359,No Dorms Available,"[Free WiFi, Free Breakfast]"
